In [3]:
'''
Author: RN
Purpose: This script reads GPS traces file, removes linear non-required points and creates a boundary
polygon used to create farmer's land boundary Polygon 
'''

################### Libraries Imports Section ############################
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, Polygon, MultiPoint
import numpy as np

################### Reading GPS Traces from/into required format ############################
df = pd.read_csv(r'C:\Users\rohan\Downloads\cordinates', names=['y','x']) #x is longitude, y is latitude
crs = {'init':'epsg:4326'}    ## CRS is assumed as WGS84 
mercatorCRS = {'init':'epsg:3857'}    ## Mercator projection for calculating final area 
geometry = [Point(xy) for xy in zip(df['x'],df['y'])]
gdf = gpd.GeoDataFrame(df, crs = crs, geometry = geometry)
gdf = gdf.reset_index()
gdf['FID'] = gdf.index
#Following line creates Point shapefile from GPS Traces for viewing purpose.Can be commented if not required.Path needs to be changed
gdf.to_file(r'C:\Users\rohan\Downloads\gps_traces_pts.shp')

##################### Calculating some statistics from Points and doing Proximity Analysis ##################################
tdf = gdf.copy()
xmin, ymin, xmax, ymax = tdf.total_bounds
xdiff = xmax - xmin
ydiff = ymax - ymin
bufferRadius = min(xdiff, ydiff)/6    ## parameter 6 is fitting as per input data here. Can be changed for different data for optimal result

tdf['geometry'] = tdf.apply(lambda row: row.geometry.buffer(bufferRadius), axis = 1)

unionPolygon =  tdf.geometry.unary_union
finalBufferedPoly = None    ###  final bigger polygon after discarding smaller isolated ones
areaOfFinalBufferedPoly = 0  ### not of much interest here apart from finding biggest polygon,hence CRS is not changed and is calculated in WGS84 only
if(unionPolygon.type=='MultiPolygon'):
    for poly in unionPolygon:
        if(poly.area > areaOfFinalBufferedPoly):
            areaOfFinalBufferedPoly = poly.area
            finalBufferedPoly = poly
else:
    finalBufferedPoly = unionPolygon
    areaOfFinalBufferedPoly = finalBufferedPoly.area

gdf['Keep'] = gdf.apply(lambda row: finalBufferedPoly.contains(row.geometry), axis = 1)

##################### Calculating and Finalizing Outputs #################################
gdf = gdf.loc[gdf.Keep==True]
outBoundaryPoly = gdf.geometry.unary_union.convex_hull
outPolyAttr = {'FID':[0]}
outPolyDf = pd.DataFrame(outPolyAttr)
geometry = [outBoundaryPoly]
outPolyGdf = gpd.GeoDataFrame(outPolyDf, crs = crs, geometry = geometry)
outPolyGdf.crs = crs 
#Following line Creates output Polygon shapefile from GPS Traces for viewing purpose.Can be commented if not required.Path needs to be changed
outPolyGdf.to_file(r'C:\Users\rohan\Downloads\farm_land_polygon.shp')